Assignment: Flowers Recognition
Dataset Description:

This dataset contains **4242** images of flowers.
The data collection is based on the data flicr, google images, yandex images.
You can use this datastet to recognize plants from the photo.

Attribute Information:<br>
The pictures are divided into **five classes**: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about **800 photos**. Photos are not high resolution, about **320x240 pixels**.<br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size.</b><br>
This is a Multiclass Classification Problem.

WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>


In [1]:
!unzip '/content/drive/MyDrive/flowers archive.zip'

In [2]:
import os
os.path.join('/content/flowers', 'flowers')  # oo interesting, you give it 2 strings and it makes a single path of them

'/content/flowers/flowers'

In [3]:
# deleting all the non-image files

In [59]:
import matplotlib
import os
import PIL


from os import listdir
from matplotlib import image
from PIL import Image
import numpy as np
import PIL.ImageOps

def create_dataset_PIL(imgfolder):
  img_data_array = []
  class_name=[]
  
  for folder in listdir(imgfolder):
    if folder != 'flowers':
      for filename in listdir(os.path.join(imgfolder, folder)):
        if '.jpg' in filename:

          # load images:
          image = Image.open(os.path.join(imgfolder , folder , filename)).resize((320,240))
          image = PIL.ImageOps.grayscale(image)
                                     #image = image.imread(os.path.join(imgfolder , folder , filename)) 
         # manipulating the image array to our needs:
          image = np.array(image)
          image = image.reshape((320*240,))
          image = image.astype('float32')
          image /= 255 

         #append data + label to their respective lists:
          img_data_array.append(image)
          class_name.append(folder)
        else : 
          pass


  return img_data_array , class_name

In [60]:
image_data , labels = create_dataset_PIL('/content/flowers')

In [61]:
dicti = dict([(label, index) for index, label in enumerate(np.unique(labels)) ])

In [62]:
target_labels = [dicti[labels[n]] for n in range(len(labels))]

In [63]:
len(target_labels)

4323

In [64]:
len(image_data)

4323

In [65]:
image_data[0].shape

(76800,)

In [66]:
# Shuffling our dataset:

import random

# image_data , target_labels

c = list(zip(image_data, target_labels))

random.shuffle(c)

image_data_shuffled , labels_shuffled = zip(*c)

image_data_shuffled = list(image_data_shuffled)
labels_shuffled = list(labels_shuffled)

In [67]:
labels = labels_shuffled
data = image_data_shuffled

In [68]:
# splitting data
train_len = len(labels)*6//10
test_len = len(labels) - train_len

In [69]:
train_data = data[0 : train_len].copy()
test_data = data[train_len : train_len + test_len].copy()

train_labels = labels[0 : train_len].copy()
test_labels = labels[train_len : train_len + test_len].copy()

In [70]:
# One hot encoding of labels:

from keras.utils.np_utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [71]:
train_labels.dtype

dtype('float32')

In [72]:
train_labels.shape

(2593, 5)

In [73]:
type(train_data)

list

In [74]:
# converting lists to numpy arrays
train_data = np.array(train_data, dtype = 'float32')

In [75]:
test_data = np.array(test_data, dtype = 'float32')

In [76]:
train_data.shape , test_data.shape

((2593, 76800), (1730, 76800))

In [19]:
# converting list to numpy array:  step not needed

#x_train = np.array([ a for a in train_data ], dtype = np.float32)

In [77]:
# dividing train_data in ratio of 7:3 for sake of validation

len_partial_train = len(train_data)*7//10
len_validation = train_len - len_partial_train

In [78]:
partial_train_data = train_data[0 : len_partial_train]
val_data = train_data[len_partial_train : ]

In [79]:
partial_train_labels = train_labels[0 : len_partial_train]
val_labels = train_labels[len_partial_train : ]

In [80]:
type(partial_train_labels)

numpy.ndarray

In [81]:
type(val_data)

numpy.ndarray

In [82]:
len_validation

778

In [83]:
partial_train_data.shape

(1815, 76800)

In [32]:
val_data.shape

(778, 76800, 3)

In [35]:
val_labels.shape

(778, 5)

In [36]:
partial_train_labels.shape

(1815, 5)

In [91]:
from keras import models
from keras import layers
from keras import regularizers

modelF = models.Sequential()

modelF.add(layers.Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001), input_shape = (320*240,))) 
modelF.add(layers.Dropout(0.2))
modelF.add(layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(0.001) ))
modelF.add(layers.Dropout(0.2))
modelF.add(layers.Dense(64, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
modelF.add(layers.Dropout(0.2))
modelF.add(layers.Dense(5, activation = 'softmax'))

modelF.compile(optimizer = 'rmsprop',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

In [92]:

historyF = modelF.fit(partial_train_data, 
                      partial_train_labels,
                      epochs = 10,
                      batch_size = 32,
                      validation_data = (val_data, val_labels)
                      )

Epoch 1/10
57/57 [==============================] - 31s 513ms/step - loss: 22.4288 - accuracy: 0.2291 - val_loss: 4.7014 - val_accuracy: 0.1902
Epoch 2/10
57/57 [==============================] - 29s 504ms/step - loss: 2.7307 - accuracy: 0.2111 - val_loss: 2.0419 - val_accuracy: 0.2301
Epoch 3/10
57/57 [==============================] - 29s 505ms/step - loss: 2.1780 - accuracy: 0.2362 - val_loss: 1.9038 - val_accuracy: 0.2301
Epoch 4/10
57/57 [==============================] - 29s 506ms/step - loss: 1.9613 - accuracy: 0.2258 - val_loss: 1.8513 - val_accuracy: 0.2301
Epoch 5/10
57/57 [==============================] - 29s 505ms/step - loss: 1.9280 - accuracy: 0.2178 - val_loss: 1.7572 - val_accuracy: 0.2301
Epoch 6/10
57/57 [==============================] - 29s 504ms/step - loss: 1.7848 - accuracy: 0.2102 - val_loss: 1.7358 - val_accuracy: 0.2301
Epoch 7/10
57/57 [==============================] - 29s 507ms/step - loss: 1.7690 - accuracy: 0.2436 - val_loss: 1.6693 - val_accuracy: 0.230